In [1]:
# standard imports
import urllib.request
import logging
import ssl
from urllib.request import quote,unquote
import requests
import joblib
import os
import re
import warnings
import json
import time
import argparse
import numpy as np
import openai
import tiktoken
import langchain

import pandas as pd
import concurrent.futures
from langchain.schema import Document
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
import random
# external imports
import faiss
from datetime import datetime
from sentence_transformers import SentenceTransformer
from azureml.core.model import Model
from azureml.core import Workspace
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [2]:
from azureml.core import Workspace
 
ws = Workspace.from_config()
default_keyvault = ws.get_default_keyvault()

In [3]:
from azureml.core import Workspace, Dataset
from azureml.core.model import Model

#ws = Workspace.from_config()
model = Model(ws,name="test-assist-code-search-faiss-index-test")
faiss_model_azure=model.download(exist_ok = True)

In [4]:
print(faiss_model_azure)

merged_faiss_db


In [5]:

def restructure_document_object(self, matching_docs):
    filtered_documents = []
    for row in matching_docs:
        input_doc_content = row.page_content + \
            "\n\n" + row.metadata.get('unit_test_cases')
        document_obj = {"input": row.page_content , "output":row.metadata.get('unit_test_cases')}
        filtered_documents.append(document_obj)
    return filtered_documents

In [6]:
def few_shot_prompting_template(self,examples):
    example_template = """
    Input:
        Task:
        write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

        Instructions:
        - understand the code snippet below and write unit test cases
        - consider every scenario for unit testing
        - only write code
        - no comments and explanations for the written code

        code snippet:
        ####

        {input}

        ####

        Example output format:
        //comments
        <code>
        </code>

    Output:
        {output}
    """
    example_prompt = PromptTemplate(
        input_variables=["input", "output"], template=example_template
    )
    
    prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        suffix="",
        input_variables=["input"],
    )

    examples_retreived = prompt.format(input="")
    
    return examples_retreived



In [7]:

# if len(matching_docs) == 0:
#     system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
#     value = [{"role": "System", "content": quote(system_message)},
#              {"role": "user", "content": quote(prompt)}]
    


In [8]:
examples = [
    {
        "input": "Who lived longer, Muhammad Ali or Alan Turing?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
    },
    {
        "input": "When was the founder of craigslist born?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
    },
    {
        "input": "Who was the maternal grandfather of George Washington?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
    },
    {
        "input": "Are both the directors of Jaws and Casino Royale from the same country?",
        "output": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
    }
]

In [9]:
# system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Java script source code as input and you need to generate unit test cases in Jasmine framework as output "
# value = [{"role": "System", "content": quote(system_message)},
#          {"role": "System", "content": quote(examples_retreived)},
#          {"role": "user", "content": quote(prompt)}]

In [10]:
class Inference:
    def __init__(self, model):
        self.model_path = model
        self.embeddings = SentenceTransformerEmbeddings(
            model_name="multi-qa-MiniLM-L6-cos-v1")
        self.db = FAISS.load_local(self.model_path, self.embeddings,allow_dangerous_deserialization=True)

        self.api_key_16k = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-16k-turbo-api-key")
        self.api_key = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-turbo-api-key")
        self.url = default_keyvault.get_secret("etg-aips-test-assist-ullm-gpt-35-turbo-url")

    def restructure_document_object(self, matching_docs):
        filtered_documents = []
        for row in matching_docs:
            input_doc_content = row.page_content + \
                "\n\n" + row.metadata.get('unit_test_cases')
            input_code = self.code_clean(row.page_content)
            output_code= self.code_clean(row.metadata.get('unit_test_cases'))
            document_obj = {"input": input_code , "output":output_code}
            filtered_documents.append(document_obj)
        return filtered_documents 

    def few_shot_prompting_template(self,examples):
        example_template = '''
        Input:
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            {input}

            ####

            Example output format:
            //comments
            <code>
            </code>

        Output:
            {output}
        '''
        print(type(examples[0]))
        
        examples_retreived = ""
        for i in examples:
            temp = example_template.format(input=i.get('input'),output=i.get('output')) + "\n\n"
            examples_retreived = examples_retreived + temp
            
#         example_prompt = PromptTemplate(input_variables=["input", "output"], template=example_template)
        
#         #print(example_prompt.format())
#         prompt = FewShotPromptTemplate(
#             examples=examples,
#             example_prompt=example_prompt,
#             suffix=" ",
#             input_variables=["input"],
#         )

        #examples_retreived = prompt.format(input=" ")
        print(examples_retreived)
        return examples_retreived
    
    def gpt_35_turbo_chat_completion(self,value):
        #self.allowSelfSignedHttps(True)
        #print(value)
        # value = [{"role": "user", "content": quote(prompt)}]
        # value_unquote = [{"role": "user", "content": prompt}]
        turbo_data = {
            'messages': value,
            'temperature': 0,
            'top_p': 0.1
        }
        print(str(turbo_data))
        # turbo_data_unquote = {
        #     "messages": value_unquote,
        #     'temperature': 0,
        #     'top_p': 0.1
        # }
        # print(str(turbo_data_unquote))
        #print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.api_key
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        #req = urllib.request.Request(url, turbo_body, headers)
        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            response_json = response.json()
            #response_json = json.loads(response.text)
            
            print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = '{:.8f}'.format(total_cost)
            model = response_json['model']
            status = "success"

        except requests.exceptions.RequestException as error:  # This is the correct syntax
            
            status_code = error.errno
            status = "failed"
            print(error)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message


    def gpt_35_turbo_16k_chat_completion(self,value):
        #print(value)
        turbo_data = {
            'messages': value,
            'temperature': 0,
            'top_p': 0.1
        }
        print(str(turbo_data))
        turbo_body = str.encode(json.dumps(turbo_data))
        url = self.url
        api_key = self.api_key_16k
        if not api_key:
            raise Exception("A key should be provided to invoke the endpoint")

        headers = {'Content-Type': 'application/json','api-key': api_key}

        result =""
        prompt_response=""
        tokens_count = 0
        total_cost = 0.0
        model =""
        status =""
        status_code = 0
        error_message=""

        try:
            start=datetime.now()

            #req = urllib.request.Request(url, turbo_body, headers)
            response = requests.post(url=url,headers=headers,data=turbo_body)
            status_code = response.status_code
            #response_json = json.loads(response.text)
            
            response_json = response.json()
            print(response_json)
            prompt_response = response_json['choices'][0]['message']['content']
            print(prompt_response)
            tokens_count = response_json['usage']['total_tokens']
            total_cost = (int(response_json['usage']['prompt_tokens'])/1000)*0.0005 + (int(response_json['usage']['completion_tokens'])/1000)*0.0015
            total_cost = '{:.8f}'.format(total_cost)
            model = response_json['model']
            status = "success"
            end=datetime.now()
            time_taken = round((end - start).total_seconds(),2)
            print(str(time_taken))
            
        except requests.exceptions.RequestException as error:  # This is the correct syntax
            end=datetime.now()
            time_taken = round((end - start).total_seconds(),2)
            print(str(time_taken))
            status_code = error.errno
            status = "failed"
            print(error)
            raise(error)

        return prompt_response,tokens_count,total_cost,model,status,status_code,error_message

    def num_tokens_from_string(self, string: str, encoding_name: str) -> int:
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def code_clean(self,code):
        code = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b', f"{{{'email'}}}", code)
        code = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', f"{{{'phone'}}}", code)
        code = re.sub(r'\b\d{3}[-]?\d{2}[-]?\d{4}\b', f"{{{'ssn '}}}", code)
        code = re.sub(r'\b\d{3}[ ]?\d{2}[ ]?\d{4}\b', f"{{{'ssn'}}}", code)
        code = re.sub(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', f"{{{'ip address'}}}", code)
        code = re.sub(r'//.*', '', code)
        code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
        return code

    def parallel_api_call(self,final_list):
        # Define the number of threads (adjust as needed)
        num_threads = len(final_list)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
            # Submit tasks to the executor
            futures = [executor.submit(self.gpt35turbo_langchain_generate, item) for item in final_list]
        
            # Wait for all tasks to complete
            concurrent.futures.wait(futures)
        
            # Get results
            results = [future.result() for future in futures]
            response = '\n'.join([x[0] for x in results])
            time_taken = max([x[1] for x in results])
            token_count = sum([x[2] for x in results])
            total_cost = sum([x[3] for x in results])
            model_used = ",".join(list(set([x[4] for x in results])))
            error = ",".join(list(set([x[5] for x in results])))
            error_type = ",".join(list(set([x[6] for x in results])))
            azure_time_taken = max([x[7] for x in results])
            self_calculated_token_size = sum([x[8] for x in results])
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size    
    
    def split_func(self,code):
        #code = self.code_clean(code)
        classes=code.split('\n')
        classlist=[]
        scriptlist=[]
        cla=''
        for i in range(len(classes)):
            #dividing classes
            if len(re.findall(r'Class\.create\(\)',classes[i]))>0:
                classlist.append(cla)
                cla=''
                cla+=classes[i]+'\n'
            else:
                cla+=classes[i]+'\n'
            if(i==len(classes)-1):
                classlist.append(cla)
                cla=''
        classlist.pop(0)
        final_classlist=[]
        for i in range(len(classlist)):   
            #counting number of functions in every class
            fu=re.findall(r':\s*function\s*\(',classlist[i])
            if(len(fu)>1):
                lines=classlist[i].split('\n')
                #getting all the script present before starting first function
                prefix=''
                ind=2
                for j in range(len(lines)):
                    if len(re.findall(r':\s*function\s*\(',lines[j]))>0:
                        ind=j
                        break
                    else:
                        prefix+='\n'+lines[j]
                    #not considering global variables
                temp_prefix=prefix.split('\n')
                prefix2=temp_prefix[1]+'\n'+temp_prefix[-1]
                class_name_line=temp_prefix[1].strip()
                suffix='    type: '+class_name_line.split(' ')[1]+'\n};'
                #exluding class defining code lines
                nlines=lines[ind:-2]
                func=[]
                temp=''
                br=0
                comment=False
                for j in range(len(nlines)):
                    if len(re.findall(r':\s*function\s*\(',nlines[j]))>0:
                        temp+='\n'+nlines[j]
                    if '{' in nlines[j]:
                        br+=nlines[j].count('{')
                        print(nlines[j],end='\n\n')
                    if '}' in nlines[j]:
                        br-=nlines[j].count('}')
                    if br == 0 and '},' in nlines[j]:
                        if len(re.findall(r':\s*function\s*\(',nlines[j]))==0:
                            temp+='\n'+nlines[j]
                        #adding class defining code prior to function
                        temp=prefix2+temp+'\n'+suffix
                        # temp=temp+'\n'
                        #appending all functions snippets to list
                        func.append(temp)
                        temp=''
                    if len(temp)>0:
                        if len(re.findall(r':\s*function\s*\(',nlines[j]))>0:
                            continue
                        temp+='\n'+nlines[j]
                #replacing that paticular class string with list of function strings
                for script in func:
                    final_classlist.append(script)
            else:
                final_classlist.append(classlist[i])
        return final_classlist

    def gpt35turbo_large_script_generate(self,code):
        token_count = self.num_tokens_from_string(code,"cl100k_base")
        no_list = int(np.ceil(token_count/4000))
        list_func = self.split_func(code)
        # using list comprehension 
        num_rec = int(np.ceil(len(list_func)/no_list))
        list_func_split = [list_func[i:i + num_rec] for i in range(0, len(list_func), num_rec)]
        final_list=[]
        for i in list_func_split:
            if self.num_tokens_from_string('\n'.join(i),"cl100k_base")>6000:
                token_count_s = self.num_tokens_from_string('\n'.join(i),"cl100k_base")
                no_list_s = int(np.ceil(token_count_s/4000))
                list_func_s = self.split_func('\n'.join(i))
                # using list comprehension 
                num_rec_s = int(np.ceil(len(list_func_s)/no_list_s))
                list_func_split_s = [list_func_s[i:i + num_rec_s] for i in range(0, len(list_func_s), num_rec_s)]
                for j in list_func_split_s:
                    final_list.append('\n'.join(j))
            else:
                final_list.append('\n'.join(i))

        response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size = self.parallel_api_call(final_list)
        
        return response,time_taken,token_count,total_cost,model_used,error,error_type,azure_time_taken,self_calculated_token_size


    def gpt35turbo_langchain_generate(self, code):

        start_time = datetime.now()
        response = None
        total_cost = ""
        tokens_count = ""
        model_used = ""
        error = ""
        error_type = ""
        azure_start_time = ""
        azure_end_time = ""
        input_prompt_size=0
        matching_docs=[]
        if len(code) <= 5:
            response = None
            error = "length of source code is too short to generate a response"
            error_type = "not_enough_tokens"

        elif len(code) > 5:

            #code = self.code_clean(code)
            prompt_template = PromptTemplate.from_template(
            '''
            Task:
            write unit test cases in Jasmine Framework for below servicenow script, create various test scenarios using multiple data points

            Instructions:
            - understand the code snippet below and write unit test cases
            - consider every scenario for unit testing
            - only write code
            - no comments and explanations for the written code

            code snippet:
            ####

            {code}

            ####

            Example output format:
            //comments
            <code>
            </code>

            ''')
            
            system_message = "Think of Yourself as a QA Engineer and tester. You will be provided with Javascript source code as input and you need to generate unit test cases in Jasmine framework as output "
           
            prompt = prompt_template.format(code = code)

            input_prompt_size = self.num_tokens_from_string(
                prompt, "cl100k_base")
            print("input prompt size: " + str(input_prompt_size))
            azure_start_time = datetime.now()
            if input_prompt_size < 1500:
                model_used = 'gpt-35-turbo'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)
                if len(matching_docs) == 0:
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "user", "content": quote(prompt)}] 
                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_chat_completion(value)
                else:
                    matching_docs = self.restructure_document_object(matching_docs)
                    examples_retreived = self.few_shot_prompting_template(matching_docs)
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "system", "content": quote(examples_retreived)},
                             {"role": "user", "content": quote(prompt)}]

                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_chat_completion(value)


            elif input_prompt_size >= 1500 and input_prompt_size < 6000:
                model_used = 'gpt-35-turbo-16k'

                retriever = self.db.as_retriever(
                    search_type="similarity_score_threshold", search_kwargs={
                        "score_threshold": 0.5, "k": 1})
                matching_docs = retriever.get_relevant_documents(code)
                
                if len(matching_docs) == 0:
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "user", "content": quote(prompt)}] 
                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_16k_chat_completion(value)
                else:
                    matching_docs = self.restructure_document_object(matching_docs)
                    examples_retreived = self.few_shot_prompting_template(matching_docs)
                    value = [{"role": "system", "content": quote(system_message)},
                             {"role": "system", "content": quote(examples_retreived)},
                             {"role": "user", "content": quote(prompt)}]

                    prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_16k_chat_completion(value)

            else:
                model_used = 'gpt-35-turbo-16k'
                matching_docs = []
                value = [{"role": "system", "content": quote(system_message)},
                         {"role": "user", "content": quote(prompt)}] 
                prompt_response,tokens_count,total_cost,model,status,status_code,error_message = self.gpt_35_turbo_chat_completion(value)




        azure_end_time = datetime.now()

        end_time = datetime.now()
        azure_time_taken = round(
            (azure_end_time - azure_start_time).total_seconds(),
            2)  # type: ignore
        time_taken = round((end_time - start_time).total_seconds(), 2)
        return response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken




In [11]:

small_code = '''var ImpactSPMIntegrationService = Class.create();

ImpactSPMIntegrationService.prototype = {
    // Initialize tables used in integration
    initialize: function() {
        this.SPM_ENTITY_LINKAGE_DETAIL_TABLE = "x_snc_impact_spm_entity_linkage_detail";
        this.SPM_ENTITY_DETAIL_TABLE = "x_snc_impact_spm_entity_detail";
        this.SPM_ENTITY_TABLE = "x_snc_impact_spm_entity";
        this.SPM_INTEGRATION_SETUP_TABLE = "x_snc_impact_spm_integration_setup";
        this.CUSTOMER_CONTACTS_SPM_INTEGRATION_SETUP_RELATION_TABLE = "x_snc_impact_spm_m2m_x_snc_cust_c_x_snc_impac";
    }'''

In [12]:
code = """
var TestGenerationUtil = Class.create();
TestGenerationUtil.tables = {
    test_assist_query: "x_snc_unit_test_ge_unit_test",
    test_assist_feedback: "x_snc_unit_test_ge_test_assist_feedback"
};
 
TestGenerationUtil.properties = {
    enable_test_assist: "x_snc_unit_test_ge.enable_test_assist",
    test_step_config_sys_id: "x_snc_unit_test_ge.test_step_config_sys_id",
    input_variable_sys_id: "x_snc_unit_test_ge.input_variable_sys_id"
};
 
TestGenerationUtil.event_name = "x_snc_unit_test_ge.run_test_assist_sync";
 
TestGenerationUtil.newGR = function(table_name) {
    var skipACL = true;
    return new(skipACL ? GlideRecord : GlideRecordSecure)(table_name);
};
 
TestGenerationUtil.testAssistEnabled = function() {
 
    var answer = false;
    var prop = gs.getProperty(TestGenerationUtil.properties.enable_test_assist, false) == "true" ? true : false;
    if (prop) {
        if (TestGenerationUtil.showTestAssistFunctionality()) {
            answer = true;
        }
    }
    return answer;
};
 
TestGenerationUtil.showTestAssistFunctionality = function() {
    var answer = true;
    if (!TestGenerationUtil.isTestAssistUser()) {
        answer = false;
    }
    return answer;
};
 
TestGenerationUtil.isTestAssistUser = function() {
    var answer = false;
    var roles = gs.getUser().getRoles();
    answer = (roles.indexOf('x_snc_unit_test_ge.user') > -1) || (roles.indexOf('x_snc_unit_test_ge.admin') > -1) ? true : false;
    return answer;
};
 
TestGenerationUtil.prototype = {
    initialize: function() {
        this.functionEndSyntax = '},';
        this.classEndSyntax = '};';
        this.scriptIncludeTableId = this.getTableId('sys_script_include');
    },
    getTableId: function(tableName) {
        var tableGr = new GlideRecord('sys_db_object');
        tableGr.addQuery('name', tableName);
        tableGr.query();
        if (tableGr.next()) {
            return tableGr.getUniqueValue();
        }
        return false;
    },
 
    createUnitTestRecord: function(scriptIncludeObj, sourceCode, documentRef, requestCategory) {
        try {
            var isCodeCoverageEnabled = gs.getProperty('x_snc_unit_test_ge.code_coverage.enabled');
            var atfUtil = new x_snc_unit_test_ge.UnitTestGenerationATFUtil();
            if (scriptIncludeObj == null && sourceCode == null && documentRef == null) {
                return false;
            } else {
                var unitTestGr = new GlideRecord('x_snc_unit_test_ge_unit_test');
                unitTestGr.initialize();
                if (scriptIncludeObj) {
                    var unitTestMappingRecord = atfUtil.getATFMappingRecord(scriptIncludeObj.getValue('sys_id'));
                    //gs.info("[TestAssistAjax] unitTestMappingRecord: " + unitTestMappingRecord);
                    if (unitTestMappingRecord) {
                        var deltaSourceCode = this.getDeltaSourceCode(scriptIncludeObj);
                        if (deltaSourceCode)
                            unitTestGr.setValue('source_code', deltaSourceCode);
                        else {
                            var errorMessage = "No new code changes found, all tests are up to date.";
                            gs.addErrorMessage(gs.getMessages(errorMessage));
                            return false;
                        }
                    } else {
                        unitTestGr.setValue('source_code', scriptIncludeObj.getValue('script'));
                    }
                    unitTestGr.setValue('document_id', scriptIncludeObj.getValue('sys_id'));
                    unitTestGr.setValue('reference_table', this.scriptIncludeTableId + '');
                } else if (sourceCode) {
                    unitTestGr.setValue('source_code', sourceCode);
                    unitTestGr.setValue('document_id', documentRef);
                    unitTestGr.setValue('reference_table', this.getTableId('x_snc_unit_test_ge_github_repo_request'));
 
                }
                var gdt = new GlideDateTime();
                unitTestGr.setValue('start_time', gdt);
                unitTestGr.setValue('request_category', requestCategory);
                unitTestGr.setValue('type', 'unit_test');
                unitTestGr.setValue('user', gs.getUserID());
                unitTestGr.setValue('client', gs.getProperty('instance_name'));
                unitTestGr.setValue('is_current', true);
                var rec = unitTestGr.insert();
                //gs.info("[TestAssistAjax] rec: " + rec + ", unitTestGr.isValidRecord(): " + unitTestGr.isValidRecord());
                if (rec) {
                    var existingUnitTestsGr = new GlideRecord('x_snc_unit_test_ge_unit_test');
                    existingUnitTestsGr.addQuery('document_id', unitTestGr.getValue('document_id'));
                    existingUnitTestsGr.addQuery('sys_id', '!=', unitTestGr.getUniqueValue());
                    existingUnitTestsGr.addQuery('is_current', true);
                    existingUnitTestsGr.query();
                    if (existingUnitTestsGr.hasNext()) {
                        existingUnitTestsGr.setValue('is_current', false);
                        existingUnitTestsGr.updateMultiple();
                    }
                    return unitTestGr;
                }
                return false;
            }
        } catch (e) {
            gs.error("[TestAssistAjax] createUnitTestRecord: " + e.toString());
        }
 
    },
 
    extractFunctionLineNumbers: function(code) {
        var functionRegex = /(\w+):[\s]*function\(/g;
        var lines = code.split('\n');
        var functions = [];
        var match;
 
        while ((match = functionRegex.exec(code)) !== null) {
            var functionName = match[1];
            var startLine = this.getFunctionStartLine(lines, match.index);
            //gs.info("match.index: " + match.index);
            var endLine = this.getFunctionEndLine(lines, match.index);
            functions.push({
                name: functionName,
                startLine: startLine,
                endLine: endLine
            });
        }
 
        return functions;
    },
 
    getFunctionStartLine: function(lines, index) {
        var line = 0;
        var charCount = 0;
 
        while (charCount <= index) {
            charCount += lines[line].length + 1;
            line++;
        }
 
        return line;
    },
 
    getFunctionEndLine: function(lines, index) {
        var line = 0;
        try {
            line = this.getFunctionStartLine(lines, index);
 
            while (!lines[line - 1].includes(this.functionEndSyntax)) { //&& !lines[line - 1].includes('}')
                line++;
                //gs.info("Line count: " + line);
            }
 
        } catch (e) {
            gs.error("Exception: " + e.message);
        }
 
        return line;
    },
 
    getDeltaSourceCode: function(scriptInlcudeRef) {
        var codeCoverageRecord = this.getCodeCoverageRecord(scriptInlcudeRef.getValue('sys_id'));
        if (codeCoverageRecord) {
            //gs.info("codeCoverageRecord: " + codeCoverageRecord.getValue('sys_id'));
            var coveredLines = (codeCoverageRecord.getValue('covered_lines')).split(','); //(codeCoverageRecord.getValue('lines')).split(',');
            var executableLines = this.getExecutableLineNumbers(scriptInlcudeRef.getValue('script')).split(','); //new global.TestGenerationUtilGlobal().getExecutableLines(scriptInlcudeRef.getValue('script')); //(new SNC.JSCoverageSourceParser().parse(scriptInlcudeRef.getValue('script'))).replace('[', '').replace(']', '').split(', ');
            var uncoveredLines = executableLines.filter(function(line) {
                return coveredLines.indexOf(line) == -1;
            });
 
            //gs.info("coveredLines: Length: " + coveredLines.length + "\n Line Numbers: " + coveredLines);
            //gs.info("executableLines: " + executableLines.length + "\n Line Numbers: " + executableLines);
            //gs.info("uncoveredLines: " + uncoveredLines);
            //uncoveredLines = [90, 100, 128, 129, 130];
            var functionLineNumbers = this.extractFunctionLineNumbers(scriptInlcudeRef.getValue('script'));
            //gs.info("functionLineNumbers: " + JSON.stringify(functionLineNumbers) + ", length: " + functionLineNumbers.length);
 
            var deltaFunctionsObj = [];
            var deltaFunctions = [];
            for (var i = 0; i < uncoveredLines.length; i++) {
                for (var j = 0; j < functionLineNumbers.length; j++) {
                    var obj = functionLineNumbers[j];
                    if (uncoveredLines[i] >= obj.startLine && uncoveredLines[i] <= obj.endLine) {
                        if (deltaFunctions.indexOf(obj.name) == -1) {
                            deltaFunctions.push(obj.name);
                            deltaFunctionsObj.push(obj);
                        }
                    }
                }
            }
 
            var scriptArray = (scriptInlcudeRef.getValue('script')).split("\n");
            var sourceCode = '';
 
            //gs.info("Script Line Numbers: " + scriptArray.length);
            for (var k = 0; k < deltaFunctionsObj.length; k++) {
                var functionObj = deltaFunctionsObj[k];
                sourceCode = sourceCode + (this.getSubsetArray(scriptArray, functionObj.startLine, functionObj.endLine)).join("\n").trim() + "\n";
            }
            sourceCode = this.addTemplateScript(scriptInlcudeRef, sourceCode);
            //gs.info(sourceCode);
            //this.createUnitTestRecord(null, sourceCode, null);
        } else {
            sourceCode = scriptInlcudeRef.getValue('script');
        }
 
 
        return sourceCode;
 
    },
 
    addTemplateScript: function(scriptInlcudeRef, sourceCode) {
        var serverSideScriptTemplateStart = 'var <scriptName> = Class.create();' + '\n' + '<scriptName>.prototype = {' + '\n';
        var clientSideScriptTemplateStart = 'var <scriptName> = Class.create();' + '\n' + '<scriptName>.prototype = Object.extendsObject(global.AbstractAjaxProcessor, {' + '\n';
        var clientSideScriptTemplateEnd = "type: '<scriptName>'" + "\n" + "});";
        var serverSideScriptTemplateEnd = "type: '<scriptName>'" + "\n" + "};";
 
        var isClientCallable = scriptInlcudeRef.getValue('client_callable');
        var codeTemplateStart;
        var codeTemplateEnd;
        var scriptCode;
        if (isClientCallable == true || isClientCallable == 'true') {
            codeTemplateStart = clientSideScriptTemplateStart;
            codeTemplateEnd = clientSideScriptTemplateEnd;
        } else {
            codeTemplateStart = serverSideScriptTemplateStart;
            codeTemplateEnd = serverSideScriptTemplateEnd;
        }
        scriptCode = codeTemplateStart + sourceCode + "\n" + codeTemplateEnd;
        scriptCode = (scriptCode.replaceAll('<scriptName>', scriptInlcudeRef.getValue('name')));
 
        return scriptCode;
    },
 
    getSubsetArray: function(array, startIndex, endIndex) {
        return array.slice(startIndex - 1, endIndex + 1);
    },
 
    isInRange: function(jsonArray, number) {
        for (var i = 0; i < jsonArray.length; i++) {
            var obj = jsonArray[i];
            if (number >= obj.startLine && number <= obj.endLine) {
                return true;
            }
        }
        return false;
    },
 
    getExecutableLineNumbers: function(code) {
        var lines = code.split('\n');
        var executableLineNumbers = [];
 
        var inMultilineComment = false;
 
        for (var i = 0; i < lines.length; i++) {
            var line = lines[i].trim();
            // Exclude empty lines
            if (line !== '') {
                if (!inMultilineComment) {
                    // Exclude single-line comments and multiline comment start lines
                    if (!line.startsWith('//') && !line.startsWith('/*') && line != '{' && line != '}' && line != this.functionEndSyntax && line != this.classEndSyntax) {
                        executableLineNumbers.push(i + 1);
                    }
                }
                // Check for multiline comment start and end
                if (line.includes('/*')) {
                    inMultilineComment = true;
                }
                if (line.includes('*/')) {
                    inMultilineComment = false;
                }
            }
        }
        return executableLineNumbers + "";
    },
 
    getCodeCoverageRecord: function(scriptIncludeId) {
        //var codeCoverageRec = new GlideRecord('sys_js_code_coverage');
        var codeCoverageRec = new GlideRecord('x_snc_code_coverge_coverage_file_records');
        //codeCoverageRec.addQuery('script_doc_id', scriptIncludeId);
        codeCoverageRec.addQuery('script_name', 'CONTAINS', scriptIncludeId);
        codeCoverageRec.orderByDesc('sys_created_on');
        codeCoverageRec.query();
        if (codeCoverageRec.next()) {
            return codeCoverageRec;
        }
        return false;
    },
 
    getInstance: function() {
        var instance = gs.getProperty("x_snc_unit_test_ge.instance_url");
        if (gs.nil(instance)) {
            return;
        }
        instance = instance.trim().toLowerCase();
        if (instance.indexOf("https://") == -1 && instance.indexOf("http://") == -1) {
            instance = "https://" + instance;
        }
        return instance;
    },
 
    scheduleSync: function() {
        var result = 'failed';
        try {
            this.queueEvent();
        } catch (e) {
            gs.error("[Test Assist] scheduleSync Error: " + e.toString());
            result = 'failed';
        }
        return result;
    },
 
    queueEvent: function(expiration_numeric) {
        if (expiration_numeric) {
            var gdt = new GlideDateTime();
            gdt.setNumericValue(expiration_numeric * 1000);
            gs.eventQueueScheduled(TestGenerationUtil.event_name, null, null, null, gdt);
        } else {
            gs.eventQueue(TestGenerationUtil.event_name, null, null);
        }
    },
 
    runTestAssistSync: function() {
        try {
            var instance = this.getInstance();
            if (gs.nil(instance)) {
                // Instance URL not defined
                gs.error("[Test Assist] runTestAssistSync: Instance URL not defined");
                return;
            }
 
            var payload = {};
            payload.instance_name = gs.getProperty('instance_name');
 
            // Last check date/time
            var last_check = gs.getProperty("x_snc_unit_test_ge.last_updated");
            var last_check_GDT = undefined;
            if (!gs.nil(last_check)) {
                last_check = parseInt(last_check);
                payload.last_check = last_check;
                last_check_GDT = new GlideDateTime();
                last_check_GDT.setNumericValue(parseInt(last_check));
            }
 
            // current date/time
            var currentGDT = new GlideDateTime();
            var last_check_end = parseInt(currentGDT.getNumericValue());
            payload.last_check_end = last_check_end;
 
            // Test Assist Feedback
            var test_assist_feedback = this.getTestAssistFeedback(last_check_GDT, currentGDT);
            if (test_assist_feedback) {
                payload.test_assist_feedback = test_assist_feedback;
            }
 
            var test_assist_user_action = this.getTestAssistUserAction(last_check_GDT, currentGDT);
            if (test_assist_user_action) {
                payload.test_assist_user_action = test_assist_user_action;
            }
 
            //gs.info("[Test Assist] runTestAssistSync payload: " + JSON.stringify(payload));
            var user_name = gs.getProperty('x_snc_unit_test_ge.test_assist_api_user_name');
            var user_password = gs.getProperty('x_snc_unit_test_ge.test_assist_api_user_password');
            var syncPOST = new sn_ws.RESTMessageV2("x_snc_unit_test_ge.Test Assist", "Sync");
            syncPOST.setBasicAuth(user_name, user_password);
            syncPOST.setStringParameter("instance", instance);
            syncPOST.setRequestBody(JSON.stringify(payload));
 
            var sync_response = syncPOST.execute();
            //gs.info("[Test Assist] runTestAssistSync body: " + sync_response.getStatusCode());
            var sync_result = {
                status_code: sync_response.getStatusCode(),
                ratelimit_reset: sync_response.getHeader("X-RateLimit-Reset"),
                body: JSON.parse(sync_response.getBody())
            };
 
            if ((sync_result.status_code == 429) && (sync_result.body.error.message == "Rate limit exceeded")) {
                // Rate limit exceeded
                this.queueEvent(sync_result.ratelimit_reset);
                return;
            } else if (sync_result.status_code == 200) {
                //gs.info("[Test Assist] runTestAssistSync body: " + JSON.stringify(sync_result));
                gs.setProperty("x_snc_unit_test_ge.last_updated", parseInt(currentGDT.getNumericValue() + 1000));
            }
        } catch (e) {
            gs.error("[Test Assist] runTestAssistSync Error: " + e.toString());
            result = "failed";
        }
    },
 
 
    getTestAssistUserAction: function(last_check_GDT, currentGDT) {
        var queryGR = TestGenerationUtil.newGR(TestGenerationUtil.tables.test_assist_query);
        var encodedQuery = 'progress_idISNOTEMPTY^test_created=true^sys_updated_on<' + currentGDT;
        if (!gs.nil(last_check_GDT)) {
            encodedQuery += "^sys_updated_on>=" + last_check_GDT;
        }
        queryGR.addEncodedQuery(encodedQuery);
        queryGR.query();
        if (!queryGR.hasNext()) {
            return;
        }
        var user_actions = [];
        while (queryGR.next()) {
            var test_modified = queryGR.getValue("modified");
            var test_script_modified = '';
            if (test_modified == true || test_modified == 'true') {
                test_modified = true;
                test_script_modified = queryGR.getValue("test_script_modified");
            } else {
                test_modified = false;
            }
            user_actions.push({
                progress_id: queryGR.getValue("progress_id") + '',
                test_created: true,
                manual_edits: test_modified,
                test_script_modified: test_script_modified
            });
        }
 
        return user_actions;
    },
 
    getTestAssistFeedback: function(last_check_GDT, currentGDT) {
        var queryGR = TestGenerationUtil.newGR(TestGenerationUtil.tables.test_assist_query);
        var encodedQuery = 'status=completed^progress_idISNOTEMPTY^feedbackISNOTEMPTY^sys_updated_on<' + currentGDT;
        if (!gs.nil(last_check_GDT)) {
            encodedQuery += "^sys_updated_on>=" + last_check_GDT;
        }
        queryGR.addEncodedQuery(encodedQuery);
        queryGR.query();
        if (!queryGR.hasNext()) {
            return;
        }
        var feedbacks = {
            query_records: [],
            feedback_records: {}
        };
        while (queryGR.next()) {
            var feedbackGR = queryGR.feedback.getRefRecord();
            var userGR = queryGR.user.getRefRecord();
            if (feedbackGR) {
                var query_record = {
                    progress_id: queryGR.getValue("progress_id"),
                    feedback_id: queryGR.getValue("feedback")
                };
                var feedback_record = {
                    user_rating: feedbackGR.getValue("user_rating"),
                    user_feedback: feedbackGR.getValue("user_feedback"),
                    user_name: userGR.getValue('user_name'),
                    feedback_id: query_record.feedback_id
                };
                feedbacks.query_records.push(query_record);
                feedbacks.feedback_records[query_record.feedback_id] = feedback_record;
            }
        }
 
        return feedbacks;
    },
 
    type: 'TestGenerationUtil'
};
"""

In [13]:
large_code = '''
var AutoAgentHandler = Class.create();
AutoAgentHandler.prototype = {
    initialize: function() {
        this.className = "AutoAgentHandler";
    },
    /**
     * @author Ram Krushna Mishra
     * @description internal method to create trigger
     * @task 
     * @param {<String>} <triggerScript> - <description>
     * @param {<String>} <triggerTime> - <description>
     * @param {<String>} <triggerName> - <description>
     * @returns {<String>} <sys id of the inserted record>
     */
    _createTrigger: function(triggerScript, triggerTime, triggerName) {
        if (JSUtil.nil(triggerScript) || JSUtil.nil(triggerTime) || JSUtil.nil(triggerName)) {
            gs.info("Invalid input provided while creating trigger triggerScript : " + triggerScript + ",triggerTime :" + triggerTime + ",triggerName : " + triggerName, this.className);
            return '';
        }

        var scheduler = new ScheduleOnce();
        scheduler.setLabel(triggerName);
        scheduler.script = triggerScript;
        scheduler.setTime(triggerTime);
        return scheduler.schedule();
    },
    /**
     * @author Ram Krushna Mishra
     * @description create trigger to be executed after the provided delay
     * @task
     * @param {<Glide Object>} <autoAgentConfigGr> - <GlideRecord of the auto agent configuration>
     * @param {<Glide Object>} <taskGr> - <GlideRecord of the task>
     * @param {<Sys Id>} <userId> - <Sys id of the user who intiated the transaction>
     * @param {<Boolean>} Flag to indicate that an active flow was deactivated
     * @returns {<Sys Id>} <Sys id of the created trigger record>
     */
    createTriggerWithDelay: function(autoAgentConfigGr, taskGr, userId, isFlowDeactivated) {
        if (JSUtil.nil(autoAgentConfigGr) || !autoAgentConfigGr.isValidRecord() || JSUtil.nil(taskGr)) {
            gs.info("Invalid input provided while creating trigger with delay ", this.className);
            return '';
        }
        // Calculate the next action time
        var nowDate = new GlideDateTime();
        var delay = autoAgentConfigGr.getValue('u_transfer_delay_in_seconds') ? parseInt(autoAgentConfigGr.getValue('u_transfer_delay_in_seconds')) : 0;
        //Trigger the flow immediately if active flow was deactivated or there is no transfer delay
        if (isFlowDeactivated || JSUtil.nil(delay))
            delay = 0;
        nowDate.addSeconds(delay);

        var triggerScript = "new AutoAgentHandler().executeAutoAgentFlow('" + taskGr.getUniqueValue() + "','" + taskGr.getTableName() + "','" + userId + "','" + autoAgentConfigGr.getUniqueValue() + "');",
            triggerName = AutoAgentConstants.DELAY_TRIGGER_PREFIX + taskGr.getValue('number') + '_' + gs.nowDateTime();
        //Adding logs for additonal troubleshooting
        gs.info('createTriggerWithDelay: Creating auto agent trigger with delay for taskId: ' + taskGr.getValue('number') + ' , autoAgentConfigGr : ' + autoAgentConfigGr.getValue('sys_id'), this.className);
        // Create Trigger
        return this._createTrigger(triggerScript, nowDate, triggerName);

    },

    /**
     * @author Vaibhav Kadam
     * @description create trigger to evaluate and execute auto agent flow 
     * @param {Sys id} Sys id of the task record
     * @param {String} Table Name of the task 
     * @param {Sys Id} Sys Id of the user
     * @param {Boolean} Interactive session or not
     * @returns {<sys id>} <description>
     */
    triggerAutoAgentFlow: function(taskId, tableName, userId, isInteractiveSession, isAssignedToChangedFromAA) {

        //Adding logs for additional troubleshooting
        gs.info('Inside triggerAutoAgentFlow for taskId : ' + taskId, this.className);
        if (JSUtil.nil(taskId) || JSUtil.nil(userId)) {
            gs.info("Invalid triggerAutoAgentFlow : Creating auto agent trigger for taskId: ", this.className);
            return '';
        }
        var triggerScript = "",
            triggerName = AutoAgentConstants.MAIN_TRIGGER_PREFIX + taskId,
            triggertime = new GlideDateTime();
        if (isAssignedToChangedFromAA) {
            triggerScript = "new AutoAgentHandler().cancelFlowWhenAssignedToChangesFromAA('" + taskId + "', false);";

        } else {
            triggerScript = "new AutoAgentHandler().executeAutoAgentFlow('" + taskId + "','" + tableName + "','" + userId + "');";
            if (JSUtil.notNil(isInteractiveSession) && !isInteractiveSession)
                triggertime.addSeconds(90); //Add 90 seconds delay to ensure there are no parallel updates happening on case when update is done via email reply
            else
                triggertime.addSeconds(30); //Adding 30 seconds delay to ensure there are no parallel updates happening on case
        }
        return this._createTrigger(triggerScript, triggertime, triggerName);
    },
    /**
     * @author Ram Krushna Mishra
     * @description get trasnfer delay from autoconfign record
     * @task
     * @param {<String>} <autoAgentConfigId> - <sysId of the auto agent configuration>
     * @returns {<String>} <transfer delay in seconds>
     */
    _getAutoAgentConfigRecord: function(autoAgentConfigId) {
        var autoAgentConfigDetails = new AutoAgentDao(AutoAgentConstants.CONFIGURATION_TABLE).getGlideRecordById(autoAgentConfigId);
        var autoAgentConfigGr = JSUtil.notNil(autoAgentConfigDetails.record) ? autoAgentConfigDetails.record : null;
        if (JSUtil.nil(autoAgentConfigGr)) {
            gs.info("Not able to fetch glide record with  autoAgentConfigId : " + autoAgentConfigId, this.className);
            return null;
        }
        return autoAgentConfigGr;
    },
    /**
     * @author Ram Krushna Mishra
     * @description get reference to auto agent definitions by order
     * @task
     * @param {<String>} <autoAgentConfigId> - <sysId of the auto agent configuration>
     * @returns {<GlideRecord>} <Reference to Auto Agent Definitions>
     */
    _getAutoAgentDefinitions: function(autoAgentConfigId) {
        if (JSUtil.nil(autoAgentConfigId))
            return null;

        var definitionQuery = 'u_auto_agent_configuration=' + autoAgentConfigId + '^u_active=true';
        var availabilityDetails = new AutoAgentDao(AutoAgentConstants.AVAILABILITY_TABLE).getGlideRecordByEncodedQuery(definitionQuery, 'u_order', false);
        var availabilityGr = JSUtil.notNil(availabilityDetails.record) ? availabilityDetails.record : null;
        if (JSUtil.nil(availabilityGr)) {
            gs.info("Not able to fetch availability records with  autoAgentConfigId : " + autoAgentConfigId, this.className);
            return null;
        }
        return availabilityGr;
    },
    /**
     * @author Ram Krushna Mishra
     * @description Execute Auto agent flow and evaluate the entry condition again and assign to auto agent if true. If any global exit condition is satisfied, do not assign and end the flow.
     * @param {Sys Id} sys id of the Task record 
     * @param {String} Table name of the task record
     * @param {Sys Id} sys id of the user record
     * @param {Sys Ids} sys ids of the auto agent configuration record
     * @returns {void} 
     */
    executeAutoAgentFlow: function(taskSysId, tableName, userId, autoAgentConfigIds) {
        gs.info("Inside executeAutoAgentFlow for " + taskSysId + " and autoAgentConfigIds " + autoAgentConfigIds, this.className);
        try {
            if (JSUtil.nil(taskSysId) || JSUtil.nil(tableName)) {
                gs.info("Invalid inputs provided to executeAutoAgentFlow ", this.className);
                return null;
            }
            var taskGr = this._getTaskRec(taskSysId, tableName);
            if (JSUtil.nil(taskGr)) {
                gs.info("Unable to fetch task record to executeAutoAgentFlow ", this.className);
                return null;
            }
            if (taskGr.u_time_zone_realignment != '') {
                return null;
            }

            // If AA is disable for record make sure all executions are completed.
            if (taskGr.getValue('u_aa_status') === AutoAgentConstants.AA_STATUS.DISABLED) {
                gs.info("AutoAgent: Unable to to start due case " + taskGr.number + " status is disabled", this.className);
                this.disableAA(taskGr, false);
                return null;
            }

            var fromDelayTrigger = false; //Flag to verify if this function is called from the delay trigger code

            if (JSUtil.notNil(autoAgentConfigIds)) {
                fromDelayTrigger = true;
            }

            var exitDetails = '';
            var skipAllResult = this.getValidSkipAllScenario(taskGr);

            if (JSUtil.notNil(skipAllResult)) {
                exitDetails = this.handleSkipAllExitScenario(taskGr, skipAllResult);
                this.handleTaskAssignment(taskGr, exitDetails, userId);

                // Set AA Status as disabled for skiped tasks
                this.disableAA(taskGr, true);
                return;
            }

            //If there is any active log record, evaluate the corresponding exit condition
            var activeLogRecord = this.getActiveExecutionLog(taskGr.getUniqueValue());
            if (JSUtil.notNil(activeLogRecord))
                exitDetails = this.handleActiveLogRecords(taskGr, activeLogRecord);
            //check if there are any parallel flows to be evaluated
            if (JSUtil.notNil(exitDetails) && exitDetails.hasOwnProperty('parallel_flows') && JSUtil.notNil(exitDetails.parallel_flows)) {
                autoAgentConfigIds += exitDetails.parallel_flows;
            }
            //If there is an active log record and no parallel flows exists, then return
            if (JSUtil.notNil(exitDetails) && exitDetails.hasOwnProperty('has_active_log') && exitDetails.has_active_log && JSUtil.nil(exitDetails.parallel_flows))
                return;

            //Determine whether the active log record was deactivated or not
            var isFlowDeactivated = JSUtil.notNil(exitDetails) && exitDetails.hasOwnProperty('auto_config_id') && JSUtil.notNil(exitDetails.auto_config_id);

            // Evaluate the entry condition and assign to auto agent, if true
            var startDetails = this.evaluateEntryConditions(taskGr, autoAgentConfigIds, fromDelayTrigger, false, isFlowDeactivated);

            if (isFlowDeactivated && !startDetails.new_flow_triggered) {
                try {
                    this.handleTaskAssignment(taskGr, exitDetails, userId);
                } catch (ex) {
                    gs.info("handleExitScenario: " + ex.message + " at" + ex.stack);
                }

                // Move AA Status back to inactive and update work_notes with mention to the user
                this.handleExitUserWorknoteMention(taskGr.getUniqueValue(), taskGr.getTableName());
            }

        } catch (ex) {
            var message = ex.getMessage();
            gs.info("Exception in executeAutoAgentFlow method. Exception Details : " + message + ex.stack, this.className);
        }
    },
    /**
     * @author Lucas Caixeta
     * @description return the first skip all scenario that match for the record
     * @param <GlideRecord> taskGr - Task that will be evaluated agains the skip all conditions
     * @returns <Object|null> The skip all result or null if not found
     */
    getValidSkipAllScenario: function(taskGr) {
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord()) {
            return;
        }

        // Evaluate skip all conditions, if true then exit
        var skipAllConfigurationGr = this.getSkipAllConfigurations(taskGr.getTableName());
        if (JSUtil.notNil(skipAllConfigurationGr)) {
            return this.isValidSkipAllScenario(taskGr, skipAllConfigurationGr);
        }
        return null;
    },
    /**
     * @author Lucas Caixeta
     * @description Validate if task as any valid skip all scenario
     * @param <GlideRecord> taskGr - Task that will be evaluated agains the skip all conditions
     * @returns <boolean> true if any result is found
     */
    hasValidSkipAllScenario: function(taskGr) {
        return JSUtil.notNil(this.getValidSkipAllScenario(taskGr));
    },
    /**
     * @author Lucas Caixeta
     * @description Return the name of all matching skip all configurations for specific task
     * @param <GlideRecord> taskGr - Task that will be evaluated agains the skip all conditions
     * @returns <Object|null> The skip all result or null if not found
     */
    getAllValidSkipAllScenarioNames: function(taskGr) {
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord()) {
            return;
        }
        // Evaluate skip all conditions, if true then exit
        var names = [];
        var skipAllConfigurationGr = this.getSkipAllConfigurations(taskGr.getTableName());
        if (JSUtil.notNil(skipAllConfigurationGr)) {
            do {
                var result = this.evalCondition(taskGr, skipAllConfigurationGr.getValue('u_conditions'));
                if (result) {
                    names.push(skipAllConfigurationGr.getDisplayValue());
                }
            }
            while (skipAllConfigurationGr.next());
        }
        return names;
    },
    /**
     * @author Lucas Schirm
     * @description Enable Needs attention and add work_note with user mention to it
     * @param <GlideRecord> Task case glide record
     * @return void
     */
    handleExitUserWorknoteMention: function(taskId, tableName) {
        if (JSUtil.nil(taskId) || JSUtil.nil(tableName))
            return;

        var taskGr = new GlideRecord(tableName);
        taskGr.get(taskId);

        gs.info("handleExitScenario: Task " + taskGr.number + " was set to inactive. Adding work note with user mention.");

        // if (taskGr.getValue('assgined_to') === AutoAgentConstants.AUTOAGENT_USER)
        //     return;

        taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.INACTIVE);
        taskGr.setValue('needs_attention', true);
        taskGr.work_notes = gs.getMessage('ns.autoagent.exit.flow.notes', taskGr.getDisplayValue('assigned_to'));
        taskGr.update();
    },

    /**
     * @author Ram Krushna Mishra
     * @description Assign to the Case to Autoagent
     * @task 
     * @param {<Sys Id>} <taskId> - <time after which the trigger should get executed>
     * @param {<type>} <name> - <description>
     * @param {<type>} <name> - <description>
     * @returns {<type>} <description>
     */
    assignTaskToUser: function(taskId, tableName, userId) {
        if (JSUtil.nil(taskId) || JSUtil.nil(tableName))
            return false;

        var inputParams = {
            'assigned_to': userId
        };

        var taskGrDetails = new AutoAgentDao(tableName).updateById(inputParams, taskId);
        if (JSUtil.nil(taskGrDetails.sys_id)) {
            gs.info("Unable to assign user " + userId + " to task " + taskId + " of table " + tableName, this.className);
            return false;
        }
        return true;
    },
    /**
     * @author Ram Krushna Mishra
     * @description Retrieve Task GlideRecord from Task Id
     * @task 
     * @param {<String>} <taskId> - <sys id of the task>
     * @param {<String>} <tableName> - <table Name>
     * @returns {<GlideRecord>} <case Gliderecord>
     */
    _getTaskRec: function(taskId, tableName) {
        if (JSUtil.nil(taskId) || JSUtil.nil(tableName))
            return null;

        var taskRecDetails = new AutoAgentDao(tableName).getGlideRecordById(taskId);
        var taskGr = JSUtil.notNil(taskRecDetails.record) ? taskRecDetails.record : null;
        if (JSUtil.nil(taskGr)) {
            gs.info("Unable to fetch glide record with  taskId : " + taskId + ", tableName :" + tableName, this.className);
            return null;
        }
        return taskGr;
    },
    /**
     * @author Ram Krushna Mishra
     * @description Verify if Last Assignee is available using WFM API
     * @task 
     * @param {<autoAgentConfigId>} Sys_id of auto agent config record
     * @param {<taskId>} Sys_id of task record
     * @returns {<Boolean>} <isAvailable>
     */
    isLastAssigneeAvailable: function(autoAgentConfigId, taskGr) {

        if (JSUtil.nil(autoAgentConfigId) || JSUtil.nil(taskGr) || !taskGr.isValidRecord())
            return {
                answer: false
            };
        var periodInMins = 0,
            timeAvailableInMins = 0,
            availabilityDef = '';

        var autoAgentConfigGr = this._getAutoAgentConfigRecord(autoAgentConfigId);
        if (JSUtil.nil(autoAgentConfigGr))
            return {
                answer: false,
                errorMessage: ""
            };

        // Get the Last Assignee User ID
        var lastAssigneeUserId = taskGr.getValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]);
        if (JSUtil.nil(lastAssigneeUserId))
            return {
                answer: false,
                errorMessage: ""
            };

        var availabilityGrs = this._getAutoAgentDefinitions(autoAgentConfigId);
        do {
            var condition = availabilityGrs.getValue('u_conditions');
            if (this.evalCondition(taskGr, condition)) {
                var periodInMs = availabilityGrs.u_total_duration.getGlideObject().getNumericValue(),
                    timeAvailableInMs = availabilityGrs.u_available_duration.getGlideObject().getNumericValue();
                periodInMins = periodInMs / 60000;
                timeAvailableInMins = timeAvailableInMs / 60000;
                availabilityDef = availabilityGrs.getValue('u_name');
                break;
            }
        } while (availabilityGrs.next());

        if (periodInMins == 0 || timeAvailableInMins == 0)
            return {
                answer: false,
                errorMessage: ""
            };

        var retVal = new AutoAgentIntegrationUtil().fetchAgentAvailability(lastAssigneeUserId, timeAvailableInMins, periodInMins);
        return {
            answer: retVal.agentAvailable,
            errorMessage: retVal.errorMessage,
            statusCode: retVal.status_code,
            availabilityDefinition: availabilityDef
        };
    },

    /**
     * @author Ram Krushna Mishra
     * @description evaluate the condition
     * @task 
     * @param {<String>} <delay> - <time after which the trigger should get executed>
     * @param {<type>} <name> - <description>
     * @param {<type>} <name> - <description>
     * @returns {<type>} <description>
     */
    evalCondition: function(sourceGr, queryCond) {
        if (JSUtil.nil(sourceGr) || !sourceGr.isValidRecord() || JSUtil.nil(queryCond))
            return false;

        return GlideFilter.checkRecord(sourceGr, queryCond);
    },

    /**
     * @author Vaibhav Kadam
     * @description Get active execution log for a task
     * @param {<Sys ID>} Sys_id of the task record
     * @param {<Sys Id>} Sys_id of the auto agent configuration record
     * @return valid GlideRecord or null
     * @example 
     * @see 
     */
    getActiveExecutionLog: function(taskId, autoAgentConfigId) {
        if (JSUtil.nil(taskId))
            return null;

        var encodedQuery = 'u_execution_status=' + AutoAgentConstants.EXECUTION_STATUS.IN_PROGRESS + '^u_task=' + taskId;
        if (JSUtil.notNil(autoAgentConfigId))
            encodedQuery += '^u_entry_configuration=' + autoAgentConfigId;

        var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).getGlideRecordByEncodedQuery(encodedQuery, '	sys_created_on', false),
            executionLogGr = JSUtil.notNil(executionLogDetails.record) ? executionLogDetails.record : null;

        if (!JSUtil.nil(executionLogGr)) {
            gs.info("Found active execution log record for task " + taskId + " and auto agent config " + autoAgentConfigId, this.className);
        }
        return executionLogGr;
    },

    /**
     * @author Vaibhav Kadam
     * @description Trigger a subflow associated with the auto agent config
     * @param {Glide Record} Glide record of auto agent config record
     * @param {<Sys ID>} Sys_id of the case record
     * @return valid GlideRecord or null
     * @example 
     * @see 
     */
    triggerSubFlow: function(autoAgentConfigGr, taskSysId) {
        try {
            if (JSUtil.nil(autoAgentConfigGr) || !autoAgentConfigGr.isValidRecord() || JSUtil.nil(taskSysId)) {
                gs.info("Invalid inputs provided to trigger subflow for task ", this.className);
                return null;
            }

            //Get the name of the subflow to be triggered
            var subflowName = this._getSubFlowDetails(autoAgentConfigGr.getValue('u_subflow'));
            if (JSUtil.nil(subflowName)) {
                return null;
            }

            // SFSTRY0098670: Use new run_as configuration to start subflow
            var runAsUser = autoAgentConfigGr.getValue('u_run_as'),
                originalUser = '';
            //Impersonate as auto agent user
            if (JSUtil.notNil(runAsUser)) {
                originalUser = this._impersonate(runAsUser);
            }
            //Build the inputs to be passed to the subflow
            var flowInputs = AutoAgentConstants.SUBFLOW_COMMON_INPUTS;
            flowInputs['task_id'] = taskSysId;
            flowInputs['auto_agent_configuration_id'] = autoAgentConfigGr.getUniqueValue();

            //trigger the subflow
            var result = sn_fd.FlowAPI.getRunner()
                .subflow(subflowName)
                .inBackground()
                .withInputs(flowInputs)
                .run();
            //DeImpersonate as auto agent user
            if (JSUtil.notNil(originalUser)) {
                originalUser = this._impersonate(originalUser);
            }
            //retrun the context id of the subflow which is triggered
            return result.getContextId();
        } catch (ex) {
            var message = ex.getMessage();
            gs.info("Exception in triggerSubFlow method. Exception details : " + message + ex.stack, this.className);
        }
    },
    /**
     * @author Vaibhav Kadam
     * @description Get subflow details associated with the auto agent config
     * @param {<Sys ID>} sys_id of the subflow
     * @return name of the subflow
     * @example 
     * @see 
     */
    _getSubFlowDetails: function(subflowId) {
        if (JSUtil.nil(subflowId))
            return null;

        var subflowDetails = new AutoAgentDao(AutoAgentConstants.SYS_HUB_FLOW_TABLE).getGlideRecordById(subflowId),
            subflowGr = JSUtil.notNil(subflowDetails.record) ? subflowDetails.record : null;

        if (JSUtil.nil(subflowGr))
            return null;

        var subflowName = String(subflowGr.sys_scope.scope) + '.' + subflowGr.getValue('internal_name');
        return subflowName;
    },
    /**
     * @author Vaibhav Kadam
     * @description Impersonate as auto agent user
     * @param {<Sys ID>} sys_id of the user to be impersonated
     * @return sysid of the user that was logged in prior to impersonation
     * @example 
     * @see 
     */
    _impersonate: function(userSysId) {
        if (!userSysId) {
            return null;
        }

        var originalUser = gs.getSession().impersonate(userSysId);
        return originalUser;
    },
    /**
     * @author Vaibhav Kadam
     * @description Cancel any active subflows for a case
     * @param {<Sys Id>} Sys id of the sys_flow_context table
     * @return void
     * @example 
     * @see 
     */
    cancelSubFlow: function(flowContextId) {
        try {
            gs.info('cancelAutoAgentSubFlow: Cancelling flowContextId: ' + flowContextId, this.className);
            sn_fd.FlowAPI.cancel(flowContextId, 'Canceling Sub Flow as one of exit conditions for auto agent is met');
        } catch (ex) {
            var message = ex.getMessage();
            gs.info("Exception in cancelSubFlow method. Exception details : " + message + ex.stack, this.className);
        }
    },
    /**
     * @author Vaibhav Kadam
     * @description Update record in u_auto_agent_execution_log table
     * @param {<Sys Id>} Sys id of the task record
     * @param {<Sys Id>} sys_id of the auto agent configuration table
     * @param {object} object with key value pairs of field and field value to be updated
     * @return void
     * @example 
     * @see 
     */
    updateExecutionLog: function(taskSysId, fieldDetails, autoAgentConfigId) {
        if (JSUtil.nil(taskSysId) || JSUtil.nil(fieldDetails))
            return null;

        var executionLogId = '',
            activeExecutionLogGr = this.getActiveExecutionLog(taskSysId, autoAgentConfigId);
        if (JSUtil.notNil(activeExecutionLogGr)) {
            do {
                var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).updateById(fieldDetails, activeExecutionLogGr.getUniqueValue());
                executionLogId = executionLogDetails.sys_id;
                if (JSUtil.nil(executionLogId)) {
                    gs.info("Unable to update execution Log record for task " + taskSysId + " and fields " + JSON.stringify(fieldDetails) + " Error details : " + executionLogDetails.errorMessage, this.className);
                }
            } while (activeExecutionLogGr.next());
        }
        return executionLogId;
    },

    /**
     * @author Ragunath Sigicherla
     * @description Get Exit Configuration with Skip All as true for given table.
     * @param {<String>} TableName - Table Name for which the SkipAll need to be verified
     * @return {boolean}
     * @example 
     * @see 
     */
    getSkipAllConfigurations: function(tableName) {
        var skipAllConfigurationGr = null,
            encodedQuery = "u_active=true^u_state=approved^u_type=exit^u_skip_all=true^u_table=" + tableName,
            configDetails = new AutoAgentDao(AutoAgentConstants.CONFIGURATION_TABLE).getGlideRecordByEncodedQuery(encodedQuery, 'u_order', false);

        skipAllConfigurationGr = JSUtil.notNil(configDetails.record) ? configDetails.record : null;

        return skipAllConfigurationGr;

    },

    /**
     * @author Vaibhav Kadam
     * @description Check if Auto agent is enabled
     * @param {String} Table name of the task record
     * @return {boolean} Returns whether the auto agent flow is enabled for a specific table or not
     * @example 
     * @see 
     */
    isAutoAgentFlowEnabled: function(tableName) {
        var flowEnablementDetails = gs.getProperty('ns.autoagent.enabled');
        if (flowEnablementDetails && JSON.parse(flowEnablementDetails).hasOwnProperty(tableName))
            return JSON.parse(flowEnablementDetails)[tableName];
        else
            return false;
    },

    /**
     * @author Vaibhav Kadam
     * @description Evalaute if it is a valid scenario to trigger auto agent flow for a specific record
     * @param {Glide Object} Glide record of a task  
     * @return {boolean} Returns if it is a valid scenario to evaluate auto agent flow conditions
     * @example 
     * @see 
     */
    isValidScenario: function(taskGr) {
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord())
            return false;

        var tableName = taskGr.getTableName();
        if (!this.isAutoAgentFlowEnabled(tableName))
            return false;

        var fieldDetails = gs.getProperty('ns.autoagent.trigger.fields');
        if (JSUtil.nil(fieldDetails) || !JSON.parse(fieldDetails).hasOwnProperty(tableName))
            return false;

        var fieldsList = JSON.parse(fieldDetails)[tableName],
            encodedQuery = '';
        //Build the encoded query to check if any of the fields have changed
        fieldsList.forEach(function(element) {
            if (JSUtil.nil(encodedQuery))
                encodedQuery = element + 'VALCHANGES';
            else
                encodedQuery += '^OR' + element + 'VALCHANGES';
        });

        if (JSUtil.nil(encodedQuery))
            return false;

        var isChildOfMajorCase = false,
            parentId = taskGr.getValue('parent');

        if (JSUtil.notNil(parentId))
            isChildOfMajorCase = this._isChildOfMajorCase(parentId);

        var result = this.evalCondition(taskGr, encodedQuery) && !isChildOfMajorCase;
        //Adding additional logs for debugging
        gs.info('AutoAgent:isValidScenario for task ' + taskGr.number + ' - Result: ' + result, this.className);
        return result;
    },

    /**
     * @author Vaibhav Kadam
     * @description Validate if case's parent is a major case or not
     * @param {Sys Id} sys_id fo the task recor
     * @return {Boolean} Returns true or false
     * @example 
     * @see 
     */
    _isChildOfMajorCase: function(parentId) {
        var encodedQuery = 'sys_id=' + parentId;
        var parentCaseDetails = new AutoAgentDao(AutoAgentConstants.CASE_TABLE).getGlideRecordByEncodedQuery(encodedQuery, '', '', true);
        var parentCaseRecord = JSUtil.notNil(parentCaseDetails.record) ? parentCaseDetails.record : null;
        if (JSUtil.notNil(parentCaseRecord)) {
            return parentCaseRecord.ref_sn_customerservice_case.u_major_case;
        }
        return false;
    },

    /**
     * @author Vaibhav Kadam
     * @description Deactivate log and cancel activate subflow if any
     * @param {Sys Id} sys_id fo the task record
     * @param {Sys Id} sys_id of the configuration record
     * @param {String} Error message from WFM api call, if any
     * @return {Sys Id} Returns sys_id of the record which is updated
     * @example 
     * @see 
     */
    deactivateLog: function(taskId, exitConfigId) {
        var fields = {};
        fields["u_exit_configuration"] = exitConfigId;
        fields["u_execution_status"] = AutoAgentConstants.EXECUTION_STATUS.COMPLETED;
        return this.updateExecutionLog(taskId, fields);
    },

    /**
     * @author Vaibhav Kadam
     * @description Validate if it is a valid Skip all scenario
     * @param {Glide Record} task record
     * @param {Glide Record} Auto agent configuration record
     * @return {sys id} Returns sys id if it is a valid scenario 
     * @example 
     * @see 
     */
    isValidSkipAllScenario: function(taskGr, skipAllConfigurationGr) {
        if (JSUtil.notNil(skipAllConfigurationGr)) {
            do {
                var result = this.evalCondition(taskGr, skipAllConfigurationGr.getValue('u_conditions'));
                if (result) {
                    return skipAllConfigurationGr.getUniqueValue();
                }
            }
            while (skipAllConfigurationGr.next());
        }
        return null;
    },

    /**
     * @author Vaibhav Kadam
     * @description Hanlde Exit scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {Sys Id} sys_id of the exit configuration record
     * @return {void}
     * @example 
     * @see 
     */
    handleSkipAllExitScenario: function(taskGr, exitConfigurationId) {
        //Check if there are any active log records
        var executionLogGr = this.getActiveExecutionLog(taskGr.getUniqueValue());
        if (JSUtil.notNil(executionLogGr)) {
            return this.handleExitScenario(taskGr, executionLogGr, exitConfigurationId);
        }
        return {
            auto_config_id: '',
            execution_log_id: ''
        };
    },

    /**
     * @author Vaibhav Kadam
     * @description Hanlde Exit scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {GlideRecord} GlideRecord of the Execution Log record
     * @param {Sys Id} sys_id of the exit configuration record
     * @return {Object} object with additional exit details
     * @example 
     * @see 
     */
    handleExitScenario: function(taskGr, executionLogGr, exitConfigurationId) {
        var entryConfigId = '',
            executionLogId = '';
        if (JSUtil.notNil(executionLogGr) && executionLogGr.isValidRecord() && JSUtil.notNil(exitConfigurationId)) {
            do {
                entryConfigId = executionLogGr.getValue('u_entry_configuration');
                //Cancel Sublfow, if any
                if (JSUtil.notNil(executionLogGr.getValue('u_subflow_context'))) {
                    gs.info('handleExitScenario: cancelling the subflow context for executionLog: ' + executionLogGr.getValue('sys_id'), this.className);
                    this.cancelSubFlow(executionLogGr.getValue('u_subflow_context'));
                }
                //Deactivate execution log record
                executionLogId = this.deactivateLog(taskGr.getUniqueValue(), exitConfigurationId);
            }
            while (executionLogGr.next());
        }
        return {
            auto_config_id: entryConfigId,
            execution_log_id: executionLogId
        };
    },
    /***
     * @author Lucas Schirm
     * @description Check if assigned_to is an auto agent
     * @param {GlideRecord} - task record
     * @return boolean
     */
    isAssignedToAutoAgent: function(taskGr) {
        // if taskgr is not a valid record, return false
        if (JSUtil.nil(taskGr) || !taskGr.isValidRecord()) {
            return false;
        }

        return taskGr.assigned_to && taskGr.assigned_to.u_type != 'SNC Employee' || taskGr.assigned_to == AutoAgentConstants.AUTOAGENT_USER;
    },
    /**
     * @author Vaibhav Kadam
     * @description Hanlde user assignment scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {Object} Object with auto agent configuration sys id and execution log sys id
     * @return {void}
     * @example 
     * @see 
     */
    handleTaskAssignment: function(taskRecordGr, additionalInfo, userId) {
        var taskGr = this._getTaskRec(taskRecordGr.getUniqueValue(), taskRecordGr.getTableName());
        //Do not change the assigned to if state is closed
        if (taskGr.getValue('state') == HICaseConstants.states.CLOSED)
            return;

        // SFSTRY0098670: If assigned to is not an Auto Agent user it should ignore function
        if (!this.isAssignedToAutoAgent(taskGr)) {
            return;
        }

        //SFSTRY0042914: Do not check the availability of last assignee who is updating the case
        var last_assignee = taskGr.getValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]),
            assignee = "";
        // if (taskGr.assigned_to != AutoAgentConstants.AUTOAGENT_USER) { ////autoagent sysID
        //     taskGr.work_notes = gs.getMessage('ns.autoagent.case.not.reassigned.notes');
        //     taskGr.update();
        //     return;
        // }
        if (last_assignee == userId)
            assignee = last_assignee;
        else {
            //Get user availability
            var userAvailablityDetails = this.isLastAssigneeAvailable(additionalInfo.auto_config_id, taskGr),
                validRestCall = userAvailablityDetails.hasOwnProperty('statusCode') && AutoAgentConstants.VALID_HTTP_CODES.indexOf(userAvailablityDetails.statusCode) > -1;
            if (userAvailablityDetails.answer)
                assignee = last_assignee;
            else if (JSUtil.notNil(userAvailablityDetails.errorMessage) && JSUtil.notNil(additionalInfo.execution_log_id)) {
                var fields = {};
                if (!validRestCall)
                    fields["u_execution_status"] = AutoAgentConstants.EXECUTION_STATUS.COMPLETED_WITH_ERRORS;
                fields["u_message"] = userAvailablityDetails.errorMessage;
                var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).updateById(fields, additionalInfo.execution_log_id);
                //Assign to last assignee even in error scenario
                if (!validRestCall)
                    assignee = last_assignee;
            }
            //Update worknotes if assignee is unavailable and it is a valid rest call
            if (userAvailablityDetails.hasOwnProperty('availabilityDefinition') && !userAvailablityDetails.answer && validRestCall) {
                this._updateTaskWorkNotes(taskGr, userAvailablityDetails.availabilityDefinition);
            }
        }
        //Assign back to the user
        this.assignTaskToUser(taskGr.getUniqueValue(), taskGr.getTableName(), assignee);
    },

    /**
     * @author Vaibhav Kadam
     * @description Hanlde user assignment scenario for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {GlideRecord} GlideRecord of the execution log table
     * @return {Object} Object with additional exit details
     * @example 
     * @see 
     */
    handleActiveLogRecords: function(taskGr, executionLogGr) {
        var exitDetails = {},
            hasActiveRecord = false,
            activeCount = 0,
            parallelFlowList = [];
        if (JSUtil.notNil(taskGr) && taskGr.isValidRecord() && JSUtil.notNil(executionLogGr) && executionLogGr.isValidRecord()) {
            do {
                activeCount++;
                //Evaluate exit configuration associated with the Entry configuration
                var isValidExit = false,
                    exitConditionQuery = String(executionLogGr.u_entry_configuration.u_exit_configuration.u_conditions);
                if (JSUtil.notNil(exitConditionQuery))
                    isValidExit = this.evalCondition(taskGr, exitConditionQuery);
                if (isValidExit) {
                    gs.info("Found valid exit for task " + taskGr.number + " with entry config " + executionLogGr.u_entry_configuration.getDisplayValue(), this.className);
                    //Handle exit scenario
                    exitDetails = this.handleExitScenario(taskGr, executionLogGr, String(executionLogGr.u_entry_configuration.u_exit_configuration));
                    activeCount--;
                } else {
                    if (JSUtil.notNil(String(executionLogGr.u_entry_configuration.u_parallel_flows)))
                        parallelFlowList.push(String(executionLogGr.u_entry_configuration.u_parallel_flows));
                }
            } while (executionLogGr.next());
        }

        exitDetails.parallel_flows = parallelFlowList.toString();
        exitDetails.has_active_log = activeCount > 0 ? true : false;
        return exitDetails;
    },

    /**
     * @author Vaibhav Kadam
     * @description Evaluate Entry config conditions for a task record
     * @param {GlideRecord} GlideRecord of the task record
     * @param {String} List of auto agent configuration ids
     * @param {Boolean} Flag to determine if this is triggered from sys_trigger record logic
     * @param {Boolean} Flag to determine if this is triggered from parallel flow logic
     * @param {Boolean} Flag to determine if any active flow was deactivated
     * @return {Sys Id} Sys Id of the user record
     * @example 
     * @see 
     */
    evaluateEntryConditions: function(taskGr, entryConfigIds, fromTrigger, isParallelFlow, isFlowDeactivated) {
        var taskAssignee = '',
            newFlowTriggered = false,
            isValidStart = false,
            startDetails = {},
            encodedQuery = "u_active=true^u_state=approved^u_type=entry^u_conditionsISNOTEMPTY^u_table=" + taskGr.getTableName();
        if (JSUtil.notNil(entryConfigIds) && isParallelFlow)
            encodedQuery += '^sys_idIN' + entryConfigIds;

        var configDetails = new AutoAgentDao(AutoAgentConstants.CONFIGURATION_TABLE).getGlideRecordByEncodedQuery(encodedQuery, 'u_order', false),
            entryConfigGr = JSUtil.notNil(configDetails.record) ? configDetails.record : null;
        if (JSUtil.notNil(entryConfigGr)) {
            do {
                isValidStart = this.evalCondition(taskGr, entryConfigGr.getValue('u_conditions'));
                if (isValidStart && fromTrigger) {
                    var subFlowContextId = '';
                    taskAssignee = entryConfigGr.getValue('u_auto_agent_user');
                    //Trigger flow and add an entry in execution log table if doesn't exist already
                    var activeLogRecord = this.getActiveExecutionLog(taskGr.getUniqueValue(), entryConfigGr.getUniqueValue());
                    if (JSUtil.nil(activeLogRecord)) {
                        //Trigger subflow if any
                        if (JSUtil.notNil(entryConfigGr.getValue('u_subflow'))) {
                            subFlowContextId = this.triggerSubFlow(entryConfigGr, taskGr.getUniqueValue());
                        }
                        //add an entry in execution log table
                        this._createExecutionLog(taskGr.getUniqueValue(), entryConfigGr.getUniqueValue(), subFlowContextId);

                        // Update AA Status to active state
                        this.setAAStatusActive(taskGr);
                    } else
                        newFlowTriggered = true; //this will help to check when an active record already exists

                } else if (isValidStart) {
                    this.createTriggerWithDelay(entryConfigGr, taskGr, gs.getUserID(), isFlowDeactivated);
                }

                if (isValidStart & JSUtil.nil(entryConfigGr.u_parallel_flows)) {
                    gs.info("Found valid start for task " + taskGr.number + " with entry config " + entryConfigGr.getDisplayValue(), this.className);
                    break;
                }
                //Handle parallel flows, if any
                if (isValidStart & JSUtil.notNil(entryConfigGr.u_parallel_flows)) {
                    startDetails = this.handleParallelFlows(taskGr, entryConfigGr.u_parallel_flows);
                    taskAssignee = startDetails.task_assignee;
                    if (!newFlowTriggered)
                        newFlowTriggered = startDetails.new_flow_triggered;
                    break;
                }
            }
            while (entryConfigGr.next());
            //Assign the task to auto agent
            if (JSUtil.notNil(taskAssignee) && taskGr.getValue('assigned_to') != taskAssignee)
                this.assignTaskToUser(taskGr.getUniqueValue(), taskGr.getTableName(), taskAssignee);
            // Based on multiple field updates we might enter into this logic more than once. We need to set the flag only when an active flow is deactivated and a new one is triggered
            if (!newFlowTriggered)
                newFlowTriggered = isValidStart && isFlowDeactivated;
        }
        return {
            task_assignee: taskAssignee,
            new_flow_triggered: newFlowTriggered
        };
    },
    /**
     * @author Vaibhav Kadam
     * @description Handle parallel flows to be executed
     * @param {GlideRecord} GlideRecord of the task record
     * @param {String} List of auto agent configuration ids
     * @return {Sys Id} Sys Id of the user record
     * @example 
     * @see 
     */
    handleParallelFlows: function(taskGr, entryConfigIds) {
        var entryConfigIdArr = entryConfigIds.split(","),
            startDetails = {};
        for (var i = 0; entryConfigIdArr.length; i++) {
            startDetails = this.evaluateEntryConditions(taskGr, entryConfigIdArr[i], false, true);
        }
        return startDetails;
    },

    /**
     * @author Vaibhav Kadam
     * @description Create an execution log record
     * @param {Sys Id} Sys Id of the task record
     * @param {Sys Id} Sys Id of the auto agent entry configuration
     * @return {Sys Id} Sys Id of the flow context
     * @example 
     * @see 
     */
    _createExecutionLog: function(taskId, entryConfigId, subFlowContextId) {
        var fields = {};
        fields["u_entry_configuration"] = entryConfigId;
        fields["u_execution_status"] = AutoAgentConstants.EXECUTION_STATUS.IN_PROGRESS;
        fields["u_task"] = taskId;
        fields["u_subflow_context"] = subFlowContextId;
        var executionLogDetails = new AutoAgentDao(AutoAgentConstants.EXECUTION_TABLE).insert(fields);
    },

    /**
     * @author Vaibhav Kadam
     * @description Update case worknotes
     * @param {Glide Object} Gliderecord of the task record
     * @param {String} Name of the agent availability definition configuration
     * @return {Sys Id} Sys Id of the task record
     * @example 
     * @see 
     */
    _updateTaskWorkNotes: function(taskGr, availabilityDefinition) {
        var msgArray = [];
        msgArray.push(taskGr.getDisplayValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]));
        msgArray.push(availabilityDefinition);
        var fields = {};
        fields["work_notes"] = gs.getMessage('ns.autoagent.last.assignee.unavailable.notes', msgArray);

        var executionLogDetails = new AutoAgentDao(taskGr.getTableName()).updateById(fields, taskGr.getUniqueValue());
    },

    /**
     * @author Adil J Patel
     * @description Method to get TSE's next available shift & time remaining in current shift
     * @param {String} userId
     * @return {Object} An object with 'shiftInfo' property that contains the message of TSE's availability
     * @example 
     * @see 
     */
    getAgentShiftTiming: function(userId) {
        // Error response if TSE is not available
        var errorResponse = {
            "shiftInfo": gs.getMessage('could_not_available')
        };

        try {
            if (userId == null)
                return errorResponse;
            var res = new AutoAgentIntegrationUtil().fetchAgentShiftAvailability(userId);
            var parsedResponse = JSON.parse(res);

            if (parsedResponse.status == "fail") {
                return errorResponse;
            }

            var message = "";
            var onShiftStartTime;
            var localOnShiftStartTime;
            var nextAvailableShiftStartTime;
            var localNextAvailableShiftStartTime;
            var timeRemainingInCurrentShift;
            var currentShiftDuration;

            // TSE's current shift start time
            if (parsedResponse.onShift.start_time) {
                onShiftStartTime = new GlideDateTime(parsedResponse.onShift.start_time);
                localOnShiftStartTime = onShiftStartTime.getDisplayValue();
            }

            // Time remaining in current shift
            if (parsedResponse.onShift.time_remaining) {
                timeRemainingInCurrentShift = parsedResponse.onShift.time_remaining;
            }

            // Current shift duration
            if (parsedResponse.onShift.duration_time) {
                currentShiftDuration = parsedResponse.onShift.duration_time;
            }

            // TSE's next shift start time
            if (parsedResponse.nextAvailableShift_details.start_time) {
                nextAvailableShiftStartTime = new GlideDateTime(parsedResponse.nextAvailableShift_details.start_time);
                localNextAvailableShiftStartTime = nextAvailableShiftStartTime.getDisplayValue();
            }
            if (onShiftStartTime || nextAvailableShiftStartTime) {
                if (timeRemainingInCurrentShift === currentShiftDuration && localOnShiftStartTime) { // Current shift didn't start yet
                    message = gs.getMessage('next_available') + localOnShiftStartTime;
                } else {
                    if (timeRemainingInCurrentShift > 0) {
                        var dur = new GlideDuration(timeRemainingInCurrentShift * 60 * 1000);
                        message += gs.getMessage('time_remaining') + dur.getByFormat('HH:mm');
                    }
                    if (localNextAvailableShiftStartTime) {
                        message += (message.length > 0 ? ',' : '') + gs.getMessage('next_available') + localNextAvailableShiftStartTime;
                    }
                }
            } else {
                return errorResponse;
            }
            return {
                "shiftInfo": message ? message : gs.getMessage('could_not_available')
            };
        } catch (ex) {
            gs.error(ex);
            return errorResponse;
        }

    },

    /**
     * @author Lucas Schirm
     * @description move task AA status to Active
     * @param <GlideRecord> taskGr - task that will be updated
     * @retunr void
     **/
    setAAStatusActive: function(taskGr) {
        // Check if is a valid record
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        // Only update if status is different from ACTIVE
        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.ACTIVE) {
            taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.ACTIVE);
            taskGr.update();
        }
    },

    /**
     * @author Lucas Schirm
     * @description Enable AA for case record
     * @param <GlideRecord> taskGR - Case record
     * @return void
     **/
    enableAA: function(taskGr) {
        // check if case is valid
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        // Change state of the field to Inactive, case it matches to any of the configuration Start it will change it to Active.
        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.INACTIVE) {
            var previousStatus = taskGr.getDisplayValue('u_aa_status') || AutoAgentConstants.AA_STATUS.DISABLED;
            taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.INACTIVE);
            taskGr.work_notes = gs.getMessage('ns.autoagent.aa_status.flow.notes', [gs.getUserDisplayName(), taskGr.getDisplayValue('u_aa_status'), previousStatus]);
            taskGr.update();
        }
    },

    /**
     * @author Lucas Schirm
     * @description Disable AA for case record
     * @param <GlideRecord> taskGr - Case that auto agent will be disabled
     * @return void
     **/
    disableAA: function(taskGr, skipAll) {
        // check if case is valid
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.DISABLED) {
            var message;
            var previousStatus = taskGr.getDisplayValue('u_aa_status') || AutoAgentConstants.AA_STATUS.INACTIVE;
            taskGr.setValue('u_aa_status', AutoAgentConstants.AA_STATUS.DISABLED);

            if (JSUtil.nil(skipAll)) {
                message = gs.getMessage('ns.autoagent.aa_status.flow.notes', [gs.getUserDisplayName(), taskGr.getDisplayValue('u_aa_status'), previousStatus]);
            } else {
                message = this.getBannerMessage(taskGr);
            }

            taskGr.work_notes = message;
            taskGr.update();
        }

        this.cancelFlowWhenStatusIsDisabled(taskGr);
    },

    /**
     * @author Lucas Schirm
     * @description Cancel existing flow/logs when case is disabled
     * @param <GlideRecord> caseGr - the case task sys_id
     * @param <String> reasonMessage - the worknotes message added to the task.
     **/
    cancelFlowWhenStatusIsDisabled: function(taskGr, reasonMessage) {
        // check if case is valid
        if (!taskGr || !taskGr.isValidRecord()) {
            return false;
        }

        if (taskGr.getValue('u_aa_status') !== AutoAgentConstants.AA_STATUS.DISABLED) {
            return false;
        }

        if (reasonMessage !== false) {
            reasonMessage = reasonMessage ? reasonMessage : "ns.autoagent.disabled.flow.notes";
        }

        var taskId = taskGr.getUniqueValue();

        // Reassign task to last_assignee
        var userId = taskGr.getValue(AutoAgentConstants.TASK_ASSIGNEE_FIELD_MAP[taskGr.getTableName()]);

        var triggerScript = "",
            triggerName = AutoAgentConstants.MAIN_TRIGGER_PREFIX + taskId,
            triggertime = new GlideDateTime();

        triggerScript = "new AutoAgentHandler().cancelFlowWhenAssignedToChangesFromAA('" + taskId + "', '" + userId + "', '" + reasonMessage + "');";
        return this._createTrigger(triggerScript, triggertime, triggerName);
    },

    /**
     * @author Atul Tripathi
     * @description Canceling existing flow/logs when case is assigned to agent from auto-agent.
     * @param {Sys Id} sys Id of Task 
     * @param <String> workNoteMessageId - A sys_ui_message ID that will be added to the case worknote
     * @return 
     * @example 
     * @see 
     */
    cancelFlowWhenAssignedToChangesFromAA: function(taskId, userId, workNoteMessageId) {
        if (JSUtil.nil(taskId)) {
            gs.info("Invalid cancelFlowWhenAssignedToChangesFromAA : Creating auto agent trigger for taskId: " + taskId, this.className);
            return;
        }

        var taskGr = new GlideRecord('sn_customerservice_case');
        taskGr.get(taskId);

        var executionLogGr = this.getActiveExecutionLog(taskId);
        var entryConfigId = '',
            executionLogId = '';
        if (JSUtil.notNil(executionLogGr)) {
            do {
                entryConfigId = executionLogGr.getValue('u_entry_configuration');

                //Cancel Sublfow, if any
                if (JSUtil.notNil(executionLogGr.getValue('u_subflow_context'))) {
                    gs.info('handleExitScenario: cancelling the subflow context for executionLog: ' + executionLogGr.getValue('sys_id'), this.className);
                    this.cancelSubFlow(executionLogGr.getValue('u_subflow_context'));
                }
                //Deactivate execution log record
                executionLogId = this.deactivateLog(taskId, "");
            }

            while (executionLogGr.next());

            // Only ignore message if strictly set to false
            if (workNoteMessageId !== false) {
                // If no message is passed, use default message
                var defaultMessage = 'ns.autoagent.cancel.flow.notes';
                workNoteMessageId = workNoteMessageId ? workNoteMessageId : defaultMessage;
                taskGr.work_notes = gs.getMessage(workNoteMessageId);
                taskGr.update();
            }

        }

        var exitResult = {
            auto_config_id: entryConfigId,
            execution_log_id: executionLogId
        };

        if (userId !== false) {
            this.handleTaskAssignment(taskGr, exitResult, userId);
        }
    },

    /**
     * @author Lucas Caixeta
     * @description Inactivated impacted cases when Configuration changes
     * @param <Glide Record> current - Current Auto Agent Configuration
     * @param <Object> previous - Object with previous configuration of the record
     * @param <String> previous_state - Previous state of the configuration
     * @return void
     */
    invalidateImpactedCases: function(current, previous, previous_state) {

        // Inactivate all current configuration records
        var table = current.getValue('u_table');
        var conditions = current.getValue('u_conditions');
        this.bulkUpdateAAStatus(table, conditions, AutoAgentConstants.AA_STATUS.INACTIVE);

        // If previous state of the configuration was approved, check if table or conditions change
        // If yes, inactivate cases for the previous configuration as well
        if (previous_state === 'approved') {
            var changed_fields = previous;
            var tableOrConditionChanged = false;
            for (var x in changed_fields) {
                // validate if table changed
                if (changed_fields[x].field_name === 'u_table') {
                    tableOrConditionChanged = true;
                    table = changed_fields[x].previous_value;
                }

                // validate if conditions changed.
                if (changed_fields[x].field_name === 'u_conditions') {
                    tableOrConditionChanged = true;
                    conditions = changed_fields[x].previous_value;
                }
            }

            if (tableOrConditionChanged) {
                this.bulkUpdateAAStatus(table, conditions, AutoAgentConstants.AA_STATUS.INACTIVE);
            }
        }
    },

    /***
     * @author Lucas Caixeta
     * @description Update AA for multiple records so they can be evaluated
     * @param <String> table - table name
     * @param <String> condition - the query conditions to update
     * @param <String> aa_status - the status to udpate to
     */
    bulkUpdateAAStatus: function(table, condition, aa_status) {
        var bulkUpdateGr = new GlideRecord(table);
        bulkUpdateGr.addEncodedQuery(condition);

        bulkUpdateGr.setValue('u_aa_status', aa_status);
        if (bulkUpdateGr.canWrite()) {
            return bulkUpdateGr.updateMultiple();
        }

        return false;
    },

    /**
     * @author Lucas Caixeta
     * @description Run pending scripts (ATF use only)
     * @param <Strign> sys_id - case sys_id
     **/
    ATFRunPendingScripts: function(taskId, checkIfExtraSchedule) {
        var taskGr = new GlideRecord('sn_customerservice_case');
        taskGr.get(taskId);

        if (!taskGr) {
            return;
        }

        // 		TriggerAutoAgent_4d96ea9c4779b9504832eb36736d430f
        // 		AutoAgent_CS6765320_2023-10-06 07:52:58
        //nameLIKETriggerAutoAgent_^ORnameLIKEAutoAgent_
        var grTrigger = new GlideRecord('sys_trigger');
        grTrigger.addEncodedQuery('nameLIKETriggerAutoAgent_' + taskId + '^ORnameLIKEAutoAgent_' + taskGr.number + '^state=0^ORDERBYDESCsys_created_on');
        grTrigger.query();

        while (grTrigger.next()) {
            gs.info("ATFAAstatus: run scheduled " + grTrigger.getDisplayName() + " for task " + taskGr.number);
            grTrigger.next_action.setDateNumericValue(0);
            grTrigger.update();
        }

        if (checkIfExtraSchedule) {
            gs.sleep(5000);
            return this.ATFRunPendingScripts(taskId, false);
        }

        return;
    },
    /**
     * @author Lucas Caixeta
     * @description Generate message to show on the banner / add to the work_notes
     * @param <GlideRecord> taskGr - Case Glide Record
     **/
    getBannerMessage: function(taskGr) {
        var configuration_names = this.getAllValidSkipAllScenarioNames(taskGr);

        // If any find any name, add a message
        if (configuration_names && configuration_names.length) {
            return gs.getMessage('ns.autoagent.skipall.banner', [configuration_names.join(',')]);
        }

        return;
    },

    /**
     * @author vivekrupender.gopu
     * @description Use this method to fetch any preferred message for a given case/user/flow. If no preferred message exists, default message is returned.
     * @param {String} flowKey - The unique identifier for a given AutoAgent Flow
     * @param {String} caseSysID - SysID of the case to check preferred Auto Agent response
     */
    getPreferredAutoAgentResponse: function(flowKey, caseSysID) {
		var caseGR = new GlideRecord('sn_customerservice_case');
        caseGR.get(caseSysID);
		
        var autoCloseUtils = new global.AutoCloseCaseUtil();
		var caseContactLanguage = autoCloseUtils.getCaseEmailLangPreference(caseGR);
		var comments = gs.getMessageLang('ns.autoagent.' + flowKey, caseContactLanguage);
        var query = 'u_auto_agent_flow=' + flowKey + '^u_case=' + caseSysID;

        if (caseGR.getValue('assigned_to') == AutoAgentConstants.AUTOAGENT_USER)
            query += '^u_user=' + caseGR.getValue('u_last_assignee');
        else
            query += '^u_user=' + caseGR.getValue('assigned_to');

        var AAResponseGR = new GlideRecord('u_auto_agent_response');
        AAResponseGR.addEncodedQuery(query);
        AAResponseGR.query();
        if (AAResponseGR.next())
            comments = '[code]' + AAResponseGR.getValue('u_preferred_message') + '[/code]';

        return comments;
    },

    type: 'AutoAgentHandler'
};
'''

In [16]:
#code = code.replace("\r\n\r\n", "\n \n").replace("\r\n", "\n").replace("\\n", "\n").replace("\n\n","\n \n")
#logger.info("Input code: \n " + str(code))
openai_flag = 'True'

model =Inference(faiss_model_azure)
code = model.code_clean(large_code)
code_token_count = model.num_tokens_from_string(code,"cl100k_base")



In [17]:
if openai_flag == 'True' and code_token_count <= 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_langchain_generate(
        code)
elif openai_flag == 'True' and code_token_count > 6000:
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_large_script_generate(
        code)    
elif openai_flag == 'False':
    response, time_taken, tokens_count, total_cost, model_used, error, error_type, azure_time_taken = model.gpt35turbo_dummy(code)

    initialize: function() {

    _createTrigger: function(triggerScript, triggerTime, triggerName) {

        if (JSUtil.nil(triggerScript) || JSUtil.nil(triggerTime) || JSUtil.nil(triggerName)) {

    createTriggerWithDelay: function(autoAgentConfigGr, taskGr, userId, isFlowDeactivated) {

        if (JSUtil.nil(autoAgentConfigGr) || !autoAgentConfigGr.isValidRecord() || JSUtil.nil(taskGr)) {

    triggerAutoAgentFlow: function(taskId, tableName, userId, isInteractiveSession, isAssignedToChangedFromAA) {

        if (JSUtil.nil(taskId) || JSUtil.nil(userId)) {

        if (isAssignedToChangedFromAA) {

        } else {

    _getAutoAgentConfigRecord: function(autoAgentConfigId) {

        if (JSUtil.nil(autoAgentConfigGr)) {

    _getAutoAgentDefinitions: function(autoAgentConfigId) {

        if (JSUtil.nil(availabilityGr)) {

    executeAutoAgentFlow: function(taskSysId, tableName, userId, autoAgentConfigIds) {

        try {

            if (JSUtil.nil(taskSysId) || JSUtil.nil(tab

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [27]:
start = time.time()
time.sleep(10)
end = time.time()
(end - start)

10.010379076004028